## Fosberg Fire Weather Index

This notebook briefly walks through how to calculate the wildfire metric `change in annual median # of days with FFWI greater than 50` from Cal-Adapt: Analytics Engine data. This notebook may be expanded upon for inclusion in cae-notebooks in the future. 

**Order of operations**:
1. Read data in
2. Calculate delta signal
3. Reprojection to census tracts
4. Min-max standardization
5. Export data
6. Generate metadata (via Cal-CRAI environment, not AE)

### Step 0: Import libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

import pyproj
import rioxarray as rio
import xarray as xr
# projection information
import cartopy.crs as ccrs
crs = ccrs.LambertConformal(
    central_longitude=-70, 
    central_latitude=38, 
    false_easting=0.0, 
    false_northing=0.0,  
    standard_parallels=[30, 60], 
    globe=None,
)

import os
import sys
sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import upload_csv_aws, pull_nc_from_directory
from scripts.utils.write_metadata import append_metadata

In [2]:
def reproject_to_tracts(ds_delta, ca_boundaries):
    df = ds_delta.to_dataframe().reset_index()
    gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.x,df.y))
    gdf = gdf.set_crs(crs)
    gdf = gdf.to_crs(ca_boundaries.crs)
    
    ca_boundaries = ca_boundaries.set_index(['GEOID'])
    
    clipped_gdf = gpd.sjoin_nearest(ca_boundaries, gdf, how='left')
    clipped_gdf = clipped_gdf.drop(['index_right'], axis=1)
    clipped_gdf = clipped_gdf.reset_index()[
        ["GEOID",f"{ds_delta.name}","geometry"]]
    ### some coastal tracts do not contain any land grid cells ###
    ### due to the WRF's underlying surface type for a given grid cell. ###
    
    # aggregate the gridded data to the tract level
    clipped_gdf_diss = clipped_gdf.reset_index().dissolve(
        by='GEOID', aggfunc='mean')
    clipped_gdf_diss = clipped_gdf_diss.rename(
        columns={f"{ds_delta.name}_right":
                 ds_delta.name}
    )
    
    # separate tracts with data from tracts without data
    clipped_gdf_nan = clipped_gdf_diss[np.isnan(
        clipped_gdf_diss[ds_delta.name]
    )]
    clipped_gdf_nan = clipped_gdf_nan[["geometry",ds_delta.name]]
    clipped_gdf_valid = clipped_gdf_diss[~np.isnan(
        clipped_gdf_diss[ds_delta.name]
    )]
    clipped_gdf_valid = clipped_gdf_valid[["geometry",ds_delta.name]]

    # compute the centroid of each tract
    clipped_gdf_nan["centroid"] = clipped_gdf_nan.centroid
    clipped_gdf_nan = clipped_gdf_nan.set_geometry("centroid")
    clipped_gdf_valid["centroid"] = clipped_gdf_valid.centroid
    clipped_gdf_valid = clipped_gdf_valid.set_geometry("centroid")
    
    # fill in missing tracts with values from the closest tract
    # in terms of distance between the tract centroids
    clipped_gdf_filled = clipped_gdf_nan.sjoin_nearest(clipped_gdf_valid, how='left')
    clipped_gdf_filled = clipped_gdf_filled[["geometry_left",f"{ds_delta.name}_right"]]
    clipped_gdf_filled = clipped_gdf_filled.rename(columns={
        "geometry_left":"geometry", f"{ds_delta.name}_right":ds_delta.name
    })
    clipped_gdf_valid = clipped_gdf_valid.drop(columns="centroid")
 
    # concatenate filled-in tracts with the original tract which had data
    gdf_all_tracts = pd.concat([clipped_gdf_valid,clipped_gdf_filled])

    return gdf_all_tracts

def min_max_standardize(df, col):
    '''
    Calculates min and max values for specified columns, then calculates
    min-max standardized values.

    Parameters
    ----------
    df: DataFrame
        Input dataframe   
    cols_to_run_on: list
        List of columns to calculate min, max, and standardize
    '''
    max_value = df[col].max()
    min_value = df[col].min()

    # Get min-max values, standardize, and add columns to df
    prefix = col # Extracting the prefix from the column name
    df[f'{prefix}_min'] = min_value
    df[f'{prefix}_max'] = max_value
    df[f'{prefix}_min_max_standardized'] = ((df[col] - min_value) / (max_value - min_value))

    # checker to make sure new min_max column values arent < 0 > 1
    df[f'{prefix}_min_max_standardized'].loc[df[f'{prefix}_min_max_standardized'] < 0] = 0
    df[f'{prefix}_min_max_standardized'].loc[df[f'{prefix}_min_max_standardized'] > 1] = 1

    # Drop the original columns -- we want to keep as a check
    # df = df.drop(columns=[col])
     
    return df

### Step 1: Read in intermediary files
These files were generated via the `climate_ae_wildfire_intermediary_data_generation.ipynb` notebook on the Analytics Engine. 

In [ ]:
# historical baseline (1981-2010) data
# download intermediary files
pull_nc_from_directory("2a_subset/climate_risk/wildfire/exposure/ffwi_hist_9km.nc", "ffwi_hist_9km.nc")

# open files
ds_hist = xr.open_dataset('ffwi_hist_9km.nc')
ds_hist

In [ ]:
# 2degC WL data -- each model is stored in a separate file
# download intermediary files
pull_nc_from_directory("2a_subset/climate_risk/wildfire/exposure/ffwi_proj_9km_ECEarth3.nc", "ffwi_proj_9km_ECEarth3.nc")
pull_nc_from_directory("2a_subset/climate_risk/wildfire/exposure/ffwi_proj_9km_MIROC.nc", "ffwi_proj_9km_MIROC.nc")
pull_nc_from_directory("2a_subset/climate_risk/wildfire/exposure/ffwi_proj_9km_MPI.nc", "ffwi_proj_9km_MPI.nc")
pull_nc_from_directory("2a_subset/climate_risk/wildfire/exposure/ffwi_proj_9km_TaiESM.nc", "ffwi_proj_9km_TaiESM.nc")

# open files
ds_proj_sim1 = xr.open_dataset('ffwi_proj_9km_ECEarth3.nc')
ds_proj_sim2 = xr.open_dataset('ffwi_proj_9km_MIROC.nc')
ds_proj_sim3 = xr.open_dataset('ffwi_proj_9km_MPI.nc')
ds_proj_sim4 = xr.open_dataset('ffwi_proj_9km_TaiESM.nc')

In [ ]:
# merge all 4 simulations into a single dataset on a new dimension "simulation"
ds_proj = xr.concat([ds_proj_sim1, ds_proj_sim2, ds_proj_sim3, ds_proj_sim4], dim='simulation')
ds_proj

In [ ]:
# reduce and calculate metric
ds_proj = ds_proj.mean(['simulation'])
ds_proj

### Step 2: Calculate delta signal

In [7]:
ds_delta = ds_proj - ds_hist
ds_delta = ds_delta['Fosberg fire weather index'] 
ds_delta.name = 'change_ffwi_days'

In [ ]:
ds_delta.min().values, ds_delta.max().values, ds_delta.mean().values

In [ ]:
ds_delta.plot()

### Step 3: Reproject to census tract boundaries

In [10]:
# load in census tract shapefile
census_shp_dir = "s3://ca-climate-index/0_map_data/2021_tiger_census_tract/2021_ca_tract/" # pulling from AWS
# census_shp_dir = "2021_ca_tract/tl_2021_06_tract.shp" # local run, requires having census tracts loaded in file tree

ca_boundaries = gpd.read_file(census_shp_dir)

In [ ]:
# convert to area-preserving CRS
ca_boundaries = ca_boundaries.to_crs(crs=3310)
ffwi_df = reproject_to_tracts(ds_delta, ca_boundaries)
ffwi_df

## Fixing header output

In [ ]:
# Create a new DataFrame with the desired columns
new_columns = ['GEOID', 'geometry', 'change_ffwi_days']
ffwi_df_cleaned = pd.DataFrame(columns=new_columns)

ffwi_df_reset = ffwi_df.reset_index()

# Populate the new DataFrame with data from the existing one
ffwi_df_cleaned['GEOID'] = ffwi_df.index  # Get GEOID from the index
ffwi_df_cleaned['geometry'] = ffwi_df_reset['geometry']  # Get geometry column
ffwi_df_cleaned['change_ffwi_days'] = ffwi_df_reset['change_ffwi_days']  # Get change_ffwi_days column

# Check the new DataFrame
ffwi_df_cleaned

### Step 4: Min-max standardization

In [ ]:
## min-max standardization
data_std = min_max_standardize(ffwi_df_cleaned, col='change_ffwi_days')

### Step 5: Export final data as csv

In [14]:
# Data will be exported via pcluster run
# clean up dataframes prior to export
data_std = data_std.drop(columns=['geometry'])

In [ ]:
data_std

In [16]:
# Move a specific column to the end of the DataFrame
column_to_move = 'change_ffwi_days'  # Replace with the actual column name
columns = [col for col in data_std.columns if col != column_to_move]  # Keep all other columns
columns.append(column_to_move)  # Add the column to move to the end

# Reassign the DataFrame with the new column order
df_in = data_std[columns]

In [22]:
# export
metric_fname = 'climate_wildfire_ffwi_metric.csv'
df_in.to_csv(metric_fname, index=False)

### Step 6: Metadata generation

In [20]:
@append_metadata
def wildfire_ffwi_process(df, export=False, export_filename=None, varname=''):
    '''
    Reduces the size of the initial daily raw temperature data in order to streamline compute time.
    Transforms the raw data into the following baseline metrics:
    * change in median annual # of days with Fosberg Fire Weather index value >50
    
    Methods
    -------
    Metric is calculated with the FFWI threshold of 50 (indicating a moderate risk day).
    
    Parameters
    ----------
    df: pd.DataFrame
        Input data.
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI wildfire metric to AWS
        True = will upload resulting df containing CAL CRAI wildfire metric to AWS
    export_filename: string
        name of csv file to be uploaded to AWS
    varname: string
        Final metric name, for metadata generation
        
    Script
    ------
    Intermediary file generation: climate_ae_wildfire_intermediary_data_generation.ipynb
    Metric calculation: climate_ae_wildfire_metrics.ipynb
    
    Note
    ----
    Because the climate projections data is on the order of 2.4 TB in size, intermediary
    processed files are not produced for each stage of the metric calculation. All processing
    occurs in a single complete run in the notebook listed above.
    '''
         
    # calculate chronic with 2°C WL
    print('Data transformation: raw projections data retrieved for warming level of 2.0°C, by manually subsetting based on GWL for parent GCM and calculating 30 year average.')
    print("Data transformation: dynamically-downscaled climate data subsetted for a-priori bias-corrected models.")
    print("Data transformation: drop all singleton dimensions (scenario).")

    # calculate historical baseline
    print("Data transformation: historical baseline data retrieved for 1981-2010, averaging across models.")
    print("Data transformation: dynamically-downscaled climate data subsetted for a-priori bias-corrected models.")
    print("Data transformation: drop all singleton dimensions (scenario).")
    
    # calculate delta signal       
    print("Data transformation: number of median annual days calculated by summing the number of days per year above 50 threshold.")
    print("Data transformation: intermediary files generated of calculated metric data, due to size of input data.")
    print("Data transformation: delta signal calculated by taking difference between chronic (2.0°C) and historical baseline.")

    # reprojection to census tracts
    print("Data transformation: data transformed from xarray dataset into pandas dataframe.")
    print("Data transformation: data reprojected from Lambert Conformal Conic CRS to CRS 3857.")
    print("Data transformation: data infilling for coastal census tracts by the average of nearest valid census tract via sjoin.nearest") ## confirm
        
    # min-max standardization
    print("Data transformation: data min-max standardized with min_max_standardize function.")
    
    if export == True:
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [df]
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{df} uplaoded to AWS.')

    if os.path.exists(df):
        os.remove(df)

In [ ]:
varname = 'climate_caladapt_wildfire_ffwi'
filename = 'climate_wildfire_ffwi_metric.csv'
wildfire_ffwi_process(filename, export=True, export_filename=None, varname='test')